# Lib Imports

In [1]:
import vtk
import pandas as pd
import numpy as np
from vtk.util.numpy_support import numpy_to_vtk

import vtkmodules.vtkInteractionStyle
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkFiltersSources import vtkConeSource
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderer
)


# Initial Variables Definitions and Data Read

In [2]:
#Define Colors
colors = vtkNamedColors()

# Load the NOx data
nox_data = pd.read_csv('../../NOx/B_NOx_12.dat', sep='\s+')



# Pollution Image representation

In [3]:
# Determine the grid dimensions
min_x = nox_data['x'].min()
min_y = nox_data['y'].min()
max_x = nox_data['x'].max()
max_y = nox_data['y'].max()

width = int((max_x - min_x) / 3) + 1
height = int((max_y - min_y) / 3) + 1

# Determine the spacing
spacing_x = (max_x - min_x) / (width - 1)
spacing_y = (max_y - min_y) / (height - 1)

# Create an image data object
image_data = vtk.vtkImageData()
image_data.SetDimensions(width, height, 1)
image_data.SetSpacing(spacing_x, spacing_y, 1) # Set the spacing here
image_data.SetOrigin(min_x, min_y, 0) # Set the origin here
image_data.AllocateScalars(vtk.VTK_FLOAT, 1)

for index, row in nox_data.iterrows():
    i = int((row['x'] - min_x) / 3)
    j = int((row['y'] - min_y) / 3)
    concentration = row['conc']
    image_data.SetScalarComponentFromFloat(i, j, 0, 0, concentration)


# Lookup Tables for Reversion of Color Gradient

#### Reversion of Default Table

In [4]:
# Define lookup table
lookupTable = vtk.vtkLookupTable()
#lookup_table.SetTableRange(min_concentration, max_concentration) #Set values if want to visualise for a certain spectrum
lookupTable.SetNumberOfTableValues(256) # The number can be altered for a smoother gradient
lookupTable.Build()

# Reverse the lookup table
for i in range(256):
    lookupTable.SetTableValue(i, *lookupTable.GetTableValue(255-i))



#### Visualisation with specified number of colors

In [5]:
ctf = vtk.vtkColorTransferFunction()
ctf.AddRGBPoint(0, 0, 0, 0)
ctf.AddRGBPoint(1, 1, 1, 0)
ctf.AddRGBPoint(2, 1, 0, 0)
#The first parameter decides which of the two will represent the color for the highest density region meanwhile the other 3 decide on the color chosen
#The higher the id (first parameter) of the colors, the higher the concentration it will be represention, with the highest accounting for the densest region
#There can be as many colors as the users want as long they are defined in new lines by incrementing the value of the first parameter

#Color Codes: 
# (x, 0, 0, 1) - blue
# (x, 0, 1, 0) - green
# (x, 1, 1, 0) - yellow
# (x, 1, 0, 0) - red
# (x, 1, 1, 1) - white
# (x, 0, 0, 0) - black


2

# Renderer Initialization

In [6]:
# Initialize the renderer
renderer = vtk.vtkRenderer()
#renderer.SetBackground(colors.GetColor3d('LightBlue')) #Background color makes the visualization more unclear


# Glyph Definitions and Pollution Mapper

In [7]:
# Create a color mapper
color_mapper = vtk.vtkImageMapToColors()
color_mapper.SetLookupTable(ctf)
color_mapper.SetInputData(image_data)
color_mapper.Update()

# Create an image actor
image_actor = vtk.vtkImageActor()
image_actor.GetMapper().SetInputData(color_mapper.GetOutput())

# Add the image actor to the renderer
renderer.AddActor(image_actor)


# Legend

In [8]:
# Create a scalar bar actor
scalar_bar = vtk.vtkScalarBarActor()
scalar_bar.SetLookupTable(ctf) # Use here the object created for the lookup table
scalar_bar.SetTitle("Pollution Concentration")
scalar_bar.SetNumberOfLabels(3) # Adjust as to provide the number of labels in the table
scalar_bar.SetLabelFormat("%.2f") # Adjust to format the concentration values

#Position and size properties
scalar_bar.SetPosition(0.1, 0.01)
scalar_bar.SetWidth(0.4)
scalar_bar.SetHeight(0.1)

# Add the scalar bar to the renderer
renderer.AddActor2D(scalar_bar)


# Create Render Window

In [9]:

# Create a render window and add the renderer to it
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)
render_window.SetSize(900, 600)
render_window.SetWindowName('25 Abril Maquete com poluicao registada a x horas')

# Create Interactor

In [10]:
# Create a render window interactor and set the render window for it
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)


# Start the visualization
interactor.Initialize()
interactor.Start()